# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#Defining CSV Path
path="../output_data/city.csv"
weather_data = pd.read_csv(path)
weather_data.head()

,City Name,Lat,Lng,Humidity,Max Temp,Country,Wind Speed,Cloudiness,Date
0,Takoradze,4.88,-1.76,87,77.0,GH,10.98,98,1592704434
1,Yulara,-25.24,130.99,37,55.4,AU,10.29,0,1592704216
2,Victoria,-4.62,55.45,83,80.6,SC,10.67,75,1592704434
3,Anloga,5.79,0.90,94,75.2,GH,4.70,20,1592704434
4,Albany,-35.02,117.89,76,48.2,AU,5.82,0,1592704434


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#Configuring Gmaps and assigning location to a variable
gmaps.configure(api_key=g_key)
locations = weather_data[["Lat", "Lng"]].astype(float)

In [10]:
#Setting up the Humidity Heatmap and saving plots
fig = gmaps.figure(center=(2,2), zoom_level=1.75)

heat_layer = gmaps.heatmap_layer(locations, weights=weather_data["Humidity"], 
                                 max_intensity=30,
                                 point_radius = 3.0)

fig.add_layer(heat_layer)



fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Filtering Data using Loc
hotel_df=weather_data.loc[(weather_data["Max Temp"]<80)& (weather_data["Max Temp"]>70)
                         &(weather_data["Wind Speed"]<10)&(weather_data["Cloudiness"]==0)]

In [12]:
#Reseting index
hotel_df=hotel_df.reset_index()
hotel_df

,index,City Name,Lat,Lng,Humidity,Max Temp,Country,Wind Speed,Cloudiness,Date
0,72,Murgab,37.50,61.97,33,73.40,TM,6.93,0,1592704442
1,134,Cáceres,-16.07,-57.68,59,74.37,BR,4.00,0,1592704450
2,171,Yerbogachën,61.28,108.01,51,74.46,RU,6.38,0,1592704455
3,219,Omsukchan,62.53,155.80,25,73.04,RU,4.03,0,1592704461
4,334,Zhanakorgan,43.91,67.25,18,75.74,KZ,3.40,0,1592704476
5,339,Weihai,37.50,122.11,59,76.80,CN,5.59,0,1592704477
6,340,Talaya,61.38,152.75,23,70.84,RU,2.51,0,1592704477
7,350,Zhigansk,66.77,123.37,33,78.48,RU,8.05,0,1592704478
8,423,Korla,41.76,86.15,66,78.01,CN,1.01,0,1592704270
9,451,Shache,38.42,77.24,16,78.62,CN,6.40,0,1592704216


In [13]:
#Testing
hotel_df.iloc[0,3]

61.97

In [14]:
#tESTING json to figure out what to use 
target_type = "lodging"
coords = f"{str(hotel_df.iloc[0,3])},{str(hotel_df.iloc[0,4])}"
radius = 5000
params = {"location": coords,"types": target_type, "radius": radius,"key": g_key}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params)
response.json()

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 61.9640637, 'lng': 32.966551},
    'viewport': {'northeast': {'lat': 61.96532798029151,
      'lng': 32.9679304302915},
     'southwest': {'lat': 61.96263001970849, 'lng': 32.9652324697085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': '8d4a0862fd10bcd0c97f4e1541616ff3a43939d9',
   'name': 'Kuha Guba',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 2136,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101629130051830845869">Куха Губа</a>'],
     'photo_reference': 'CmRaAAAAiekduM9wv3P7Va_6RTxJIBESCSa4D5wBFBnzmAUUM-OdFzXBBLxdkQxNrtUXuuRYbZ-L4ErByiJmjpDHidBu78nvnU27PcDzFrberXYjD4UR2F7lrl_rReP3o7EgkExcEhDZw-4XcR6vMwsUUy3Vr8neGhSBvmU04eFdrmMlkRTbDdEVCdtaVQ',
     'width': 2848}],
   'place_id': 'ChIJkSWulJnuoUYR-snj23gxC_U',
   'plus_code': {'compound_code': 'XX78+JJ Сяргила́хта, Republic of Ka

In [15]:
#For Loop to get Hotel name. Used an except to single out cities that dont have a hotel name within the desired radius
Hotel=[]
for x in range(len(hotel_df["Lat"])):
    try:
        target_type = "lodging"
        coords = f"{str(hotel_df.iloc[x,2])},{str(hotel_df.iloc[x,3])}"
        radius = 5000
        params = {"location": coords,"types": target_type,"radius": radius,"key": g_key}
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params)
        Hotel.append(response.json()["results"][0]["name"])
    except:
        Hotel.append("NaN")
print(Hotel)

['NaN', 'Riviera Pantanal Hotel', 'NaN', 'NaN', '"Ақпалас" қонақ үйі', 'Beihai Travel Resort', 'NaN', 'Gostinichnyy Kompleks Aylgy', 'Gulisitan Hotel', 'Delong Hotel', 'Elite Hotel', 'Ilha Flat Hotel']


In [16]:
hotel_df=hotel_df.rename(columns={"City Name": "City"})
hotel_df

,index,City,Lat,Lng,Humidity,Max Temp,Country,Wind Speed,Cloudiness,Date
0,72,Murgab,37.50,61.97,33,73.40,TM,6.93,0,1592704442
1,134,Cáceres,-16.07,-57.68,59,74.37,BR,4.00,0,1592704450
2,171,Yerbogachën,61.28,108.01,51,74.46,RU,6.38,0,1592704455
3,219,Omsukchan,62.53,155.80,25,73.04,RU,4.03,0,1592704461
4,334,Zhanakorgan,43.91,67.25,18,75.74,KZ,3.40,0,1592704476
5,339,Weihai,37.50,122.11,59,76.80,CN,5.59,0,1592704477
6,340,Talaya,61.38,152.75,23,70.84,RU,2.51,0,1592704477
7,350,Zhigansk,66.77,123.37,33,78.48,RU,8.05,0,1592704478
8,423,Korla,41.76,86.15,66,78.01,CN,1.01,0,1592704270
9,451,Shache,38.42,77.24,16,78.62,CN,6.40,0,1592704216


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
#Added Hotel name to the sata frame and excluded cities without a hotel within a 5K radius
hotel_df["Hotel Name"]=Hotel 
hotel_df=hotel_df.loc[hotel_df["Hotel Name"]!="NaN"]
hotel_df

,index,City,Lat,Lng,Humidity,Max Temp,Country,Wind Speed,Cloudiness,Date,Hotel Name
1,134,Cáceres,-16.07,-57.68,59,74.37,BR,4.00,0,1592704450,Riviera Pantanal Hotel
4,334,Zhanakorgan,43.91,67.25,18,75.74,KZ,3.40,0,1592704476,"""Ақпалас"" қонақ үйі"
5,339,Weihai,37.50,122.11,59,76.80,CN,5.59,0,1592704477,Beihai Travel Resort
7,350,Zhigansk,66.77,123.37,33,78.48,RU,8.05,0,1592704478,Gostinichnyy Kompleks Aylgy
8,423,Korla,41.76,86.15,66,78.01,CN,1.01,0,1592704270,Gulisitan Hotel
9,451,Shache,38.42,77.24,16,78.62,CN,6.40,0,1592704216,Delong Hotel
10,484,Campo Maior,-4.83,-42.17,77,75.00,BR,3.98,0,1592704502,Elite Hotel
11,518,Ilhabela,-23.78,-45.36,64,73.40,BR,3.36,0,1592704508,Ilha Flat Hotel


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
heat = gmaps.heatmap_layer(locations, weights=hotel_df["Humidity"], 
                                 max_intensity=30,
                                 point_radius = 3.0)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(heat)


fig

# Display figure


Figure(layout=FigureLayout(height='420px'))